In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pickle
from tqdm.auto import tqdm
import json
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [3]:
from import_casa import casa
from casa import Cadet

In [4]:
cadet = Cadet.load(casa.get_data_path()/"cadet/op20.2")
cadet

[INFO] 2021-07-06 17:36:12,636 gensim.utils: loading KeyedVectors object from E:\LangOn\casa\data\cadet\op20.2\ft-2020.kv
[INFO] 2021-07-06 17:36:12,962 gensim.utils: setting ignored attribute vectors_norm to None
[INFO] 2021-07-06 17:36:12,964 gensim.utils: setting ignored attribute vectors_vocab_norm to None
[INFO] 2021-07-06 17:36:12,965 gensim.utils: setting ignored attribute vectors_ngrams_norm to None
[INFO] 2021-07-06 17:36:12,966 gensim.utils: setting ignored attribute buckets_word to None
[INFO] 2021-07-06 17:36:13,087 gensim.utils: FastTextKeyedVectors lifecycle event {'fname': 'E:\\LangOn\\casa\\data\\cadet\\op20.2\\ft-2020.kv', 'datetime': '2021-07-06T17:36:13.087609', 'gensim': '4.0.0', 'python': '3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.18362-SP0', 'event': 'loaded'}


<Cadet: FastText(9851, 200), Seeds(79)>

In [5]:
with (casa.get_data_path() / f"threads/cht2021-JanMay.pkl").open("rb") as fin:
    opdata = pickle.load(fin)

In [6]:
cadet.detect("!@3")

{'entity': ['中華電信', '台灣大哥大', '遠傳電信', '台灣之星', '亞太電信', '無框行動'],
 'entity_probs': array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667]),
 'service': [('通訊品質', '涵蓋'),
  ('資費方案', '方案活動'),
  ('加值服務', '影音娛樂'),
  ('通訊品質', '網速'),
  ('其他', '手機')],
 'service_probs': array([0.05548318, 0.05088445, 0.05004898, 0.04945965, 0.04545599]),
 'seeds': ['離島', '大4G', 'book', 'mbps', '綁約'],
 'seed_probs': array([0.01991876, 0.01826779, 0.01796786, 0.01775628, 0.01768285])}

In [7]:
opdata[0].get_opinion().id[1:9]

'20210105'

In [8]:
from datetime import datetime
datetime.strptime(opdata[0].get_opinion().id[1:9], "%Y%m%d").isocalendar()
datetime.strptime("20210101", "%Y%m%d").isocalendar()

(2020, 53, 5)

In [9]:
det_data = []
for thread_x in tqdm(opdata):
    for op in thread_x.opinions():
        if not (isinstance(op.id, str) and len(op.id)>=9): continue
        if not op.text.strip(): continue
        det = cadet.detect(op.text)        
        ent = det["entity"][0] if det["entity_probs"][0] > 0.3 else "無"
        srv = det["service"][0] if det["service_probs"][0] > 0.1 else ("無", "無")
        src = op.source
        rating = {"R": -1, "Y": 0, "G": 1}.get(op.sentence_sentiment, 0)
        date = datetime.strptime(op.id[1:9], "%Y%m%d") 
        year = date.isocalendar()[0]
        woy = date.isocalendar()[1]
        det_data.append((
            op.id, year, woy, src, ent, srv[0], srv[1], rating, op.text
        ))    

In [10]:
det_dfr = pd.DataFrame.from_records(det_data, columns="opid,year,woy,source,entity,category,attribute,rating,text".split(","))
det_dfr.to_csv(casa.get_data_path() / "casper/cadet_JanMay.csv", index=None)

In [13]:
det_dfr.loc[(det_dfr.entity=="遠傳電信") & (det_dfr.woy==19) & (det_dfr.rating!=0), :]

,opid,year,woy,source,entity,category,attribute,rating,text
85732,D202105150647574030073719504472067,2021,19,facebook粉絲團,遠傳電信,通訊品質,網速,1,我以前就是用遠傳的，結果常常收不到訊號，跑去問遠傳，它們卻說是我的手機太老，我馬上跑去中華電...
86264,D202105132356403069831755787168030,2021,19,facebook粉絲團,遠傳電信,資費方案,方案活動,1,用了近10幾年 只有一個感想 越來越爛 訊號超級差 收費還比人家貴 要不是合約還要一年 不然...
86411,D20210512031221739243217593029,2021,19,mobile01,遠傳電信,其他,客服,-1,遠傳真是欺負人 透過樓主分享的文現在才知道真相 之前我內人一直跟我抱怨遠傳晚上就是不順 三不...
87094,D20210513143917548377788058379,2021,19,BabyHome 寶貝家庭親子網,遠傳電信,加值服務,影音娛樂,1,我用了遠傳兩年多最近看別人分享才知道遠傳有心生活app換購東西或是直接兌換一些生活相關的用品...
87101,D20210513203858474222753353585,2021,19,BabyHome 寶貝家庭親子網,遠傳電信,加值服務,電信APP,1,我們全家都是遠傳，心生活的設計，對用戶真的很便利也很多元
...,...,...,...,...,...,...,...,...,...
96467,D202105151050499802906141902428149207759211391...,2021,19,TWitter搜尋,遠傳電信,通訊品質,涵蓋,-1,@IN0_0325_ 遠傳也會⋯收訊也很爛🥲
96472,D202105161050283637828432944063658495646058546...,2021,19,TWitter搜尋,遠傳電信,無,無,-1,我成功擺脫爛遠傳了！
96908,D20210515024130690300213287431,2021,19,mobile01,遠傳電信,資費方案,續約攜碼,1,遠傳到約想攜碼 可配機也能單轉門號 來信告知攜碼月租 獲配機可折多可折多少
97881,D202105151033370617044055406227938556228344487...,2021,19,TWitter搜尋,遠傳電信,加值服務,智慧音箱,-1,@IN0_0325_ @maimaimjj 我們這幾次的spaces民調下來最容易斷線的是台...


In [ ]:
det_dfr.loc[(det_dfr.entity=="中華電信") & (det_dfr.category=="加值服務") & (det_dfr.attribute=="影音娛樂") & (det_dfr.rating!=0), :]

In [4]:
cadet.detect(opdata[0].get_opinion().text)

NameError: name 'opdata' is not defined

In [34]:
opdata[0].get_opinion().text

'不知道各位馬麻有沒有人跟我一樣手機不是用吃到飽然後有時候流量又會剛好不夠用的情況～我自己是雖然平常大部分時間都在家，在家有Wifi可以用但偶爾還是會有比較常出門的時候，結果流量就被會我提早用光..今天去問家裡附近門市比較熟的店員，有沒有什麼解決方法他跟我說現在中華有一個加購上網量買一送一的活動，聽起來很適合我後來我回家又上網研究了一下還真的蠻划算的像他加購5GB上網量690元，買一送一會多給你5GB這樣算起來1GB也才69元超便宜的！重點是他是送流量兌換卷，就算你不用也可以轉送給家人朋友我是覺得用加購的方式比較有彈性很不錯，來跟各位馬麻分享一下'